In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

def check(n_point, df):
    N = n_point
    df = df
    check_constraint1 = []
    check_constraint2 = []
    check_constraint3 = []

    check_constraint1_check = False
    check_constraint2_check = False
    check_constraint3_check = False

    for k in range(len(df)):
        #c[0][0]
        for i in range(N):
            check_tempc1_1 = dimod.quicksum(df['sample'][k][f'c[{i}][{j}]'] for j in range(N) if i!=j)
            if check_tempc1_1 != 1:
                check_constraint1_check = True
        for i in range(N):
            check_tempc2_1 = dimod.quicksum(df['sample'][k][f'c[{j}][{i}]'] for j in range(N) if i!=j)
            if check_tempc2_1 != 1:
                check_constraint2_check = True
        if check_constraint1_check == True:
            check_constraint1.append(k)
            check_constraint1_check = False
        if check_constraint2_check == True:
            check_constraint2.append(k)
            check_constraint2_check = False

    num_index = []
    for i in range(len(df)):
        for c1 in check_constraint1:
            num_index.append(c1)
            #num_index = np.delete(num_index, c1)
        for c2 in check_constraint2:
            num_index.append(c2)
        for c3 in check_constraint3:
            num_index.append(c3)

    if num_index == []:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
    else:
        num_index = np.unique(num_index)
        feasible_index = np.arange(0,len(df),1)
        feasible_index = np.delete(feasible_index, num_index)

    min_energy = 1e9
    min_index = 0
    for i in feasible_index:
        if (min_energy > df['energy'][i]) and (df['energy'][i]>=0):
            min_energy = df['energy'][i]
            min_index = i
    if len(feasible_index) == 0:
        done = False#True # infeasible
    else:
        done = True #False # feasible
    #print(done)
    #print(df['energy'][min_index])
    energycal = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                    energycal += distances[i][j]*df['sample'][min_index][f'c[{i}][{k}]']*df['sample'][min_index][f'c[{j}][{(k+1)%N}]']

    success_probability = len(feasible_index)/len(df)
    return df['sample'][min_index], df['energy'][min_index], done, energycal, success_probability#df['energy'][min_index] #min_index, energycal


In [3]:
# Get the distance matrix
from scipy.spatial import distance_matrix

def TSP_instance(n_customer: int, seed: int = 0):
    """
    n_customer: 고객 수 (depot 제외). 거리행렬 크기는 (n_customer+1) x (n_customer+1)
    seed: 재현성 제어용 (원하면 N이나 run index로 바꿔도 됨)
    """
    n_point = n_customer + 1
    assert n_point >= 1

    # 재현성
    rng = np.random.RandomState(seed)

    # 무작위 좌표 생성
    df = pd.DataFrame({
        'x': rng.randint(0, 100, n_point),
        'y': rng.randint(0, 100, n_point),
    })
    # depot (index 0) 고정
    df.loc[0, 'x'] = 0
    df.loc[0, 'y'] = 0

    D = distance_matrix(df[['x', 'y']].values, df[['x', 'y']].values)
    return np.asarray(D, dtype=float)


In [4]:
from dimod import ConstrainedQuadraticModel, CQM, Binary, quicksum, BinaryQuadraticModel, ExactCQMSolver
from dwave.samplers import SteepestDescentSampler, SimulatedAnnealingSampler
import dimod
import neal
import numpy as np
#import greedy

In [5]:
from pyqubo import Array, Placeholder, Constraint

In [6]:
# ---------- QUBO build ----------
def tsp_pyqubo(n_point, distances, penalty_scale=10.0):
    x = Array.create('c', shape=(n_point, n_point), vartype='BINARY')

    city_once = 0
    for i in range(n_point):
        city_once += Constraint((sum(x[i, k] for k in range(n_point)) - 1) ** 2, label=f"city_{i}")

    pos_once = 0
    for k in range(n_point):
        pos_once += Constraint((sum(x[i, k] for i in range(n_point)) - 1) ** 2, label=f"pos_{k}")

    dist_term = 0
    for k in range(n_point):
        kp1 = (k + 1) % n_point
        for i in range(n_point):
            xi = x[i, k]
            for j in range(n_point):
                dij = distances[i][j]
                if dij != 0:
                    dist_term += dij * xi * x[j, kp1]

    A = Placeholder("A")
    H = dist_term + A * (city_once + pos_once)

    model = H.compile()
    max_d = float(np.max(distances)) if n_point > 1 else 1.0
    feed = {"A": penalty_scale * max_d}
    bqm = model.to_bqm(feed_dict=feed)
    return model, bqm, feed

In [ ]:

for n_point in range(1, 11):
    distance = TSP_instance(n_point-1)
    for j in range(1, 11):
        pyqubo_model, pyqubo_bqm, pyqubo_feed_dict = tsp_pyqubo(n_point, distances, penalty_scale=1)
        sampleset = SimulatedAnnealingSampler().sample(pyqubo_bqm, num_reads = 2000) # , num_sweeps = 2000
        samples_df = sampleset.to_pandas_dataframe(sample_column=True)
        samples_df = samples_df.sort_values(by='energy', ascending = True)
        samples_df = samples_df.reset_index(drop=True)
        best_sample, best_energy, is_feasible, route_cost, success_rate = check(n_point=n_point-1, df=samples_df)
        

In [ ]:
import math, random

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distance(self, city):
        return math.hypot(self.x - city.x, self.y - city.y)

    def __repr__(self):
        return f"({self.x}, {self.y})"


In [ ]:
# 2. City 객체로 변환
cities = [City(x, y) for x, y in zip(df['x'], df['y'])]

In [7]:
import numpy as np
import pandas as pd
from neal import SimulatedAnnealingSampler
from tqdm import tqdm
import time

rows = []

for n_point in tqdm(range(1, 11), desc="N loop"):
    N = n_point - 1
    distances = TSP_instance(N, seed=0)

    assert distances.shape == (n_point, n_point)

    best_list, feas_list, cost_list, succ_list, time_list = [], [], [], [], []

    for j in tqdm(range(10), desc=f"Runs for N={N}", leave=False):
        seed = 1000 * n_point + j

        pyqubo_model, pyqubo_bqm, pyqubo_feed_dict = tsp_pyqubo(
            n_point=n_point, distances=distances, penalty_scale=1
        )

        # ---- 시간 측정 시작 ----
        t0 = time.time()
        sampleset = SimulatedAnnealingSampler().sample(
            pyqubo_bqm, num_reads=2000, seed=seed
        )
        elapsed_ms = (time.time() - t0) * 1000  # ms 단위로 변환
        # ---- 시간 측정 끝 ----

        df = (sampleset.to_pandas_dataframe(sample_column=True)
                         .sort_values('energy', ascending=True)
                         .reset_index(drop=True))

        best_sample, best_energy, is_feasible, route_cost, success_rate = \
            check(n_point=N, df=df)

        best_list.append(float(best_energy))
        feas_list.append(1.0 if bool(is_feasible) else 0.0)
        cost_list.append(float(route_cost))
        succ_list.append(float(success_rate))
        time_list.append(elapsed_ms)

    row = {
        "N": N,
        "n_point": n_point,
        "best_energy_mean": np.mean(best_list),
        "best_energy_std":  np.std(best_list, ddof=1),
        "feasible_rate":    np.mean(feas_list),
        "route_cost_mean":  np.mean(cost_list),
        "route_cost_std":   np.std(cost_list, ddof=1),
        "success_rate_mean":np.mean(succ_list),
        "success_rate_std": np.std(succ_list, ddof=1),
        "time_mean_ms":     np.mean(time_list),
        "time_std_ms":      np.std(time_list, ddof=1),
    }
    rows.append(row)

summary_df = pd.DataFrame(rows).sort_values("N").reset_index(drop=True)
print(summary_df)

summary_df.to_csv("tsp_sa_summary_N0to9_10runs.csv", index=False)


N loop: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [02:53<00:00, 17.35s/it]

   N  n_point  best_energy_mean  best_energy_std  feasible_rate  \
0  0        1          0.000000     0.000000e+00            1.0   
1  1        2        163.682620     0.000000e+00            0.0   
2  2        3        206.911077     2.995911e-14            1.0   
3  3        4        238.059641     0.000000e+00            1.0   
4  4        5        249.433830     0.000000e+00            1.0   
5  5        6        312.493932     0.000000e+00            1.0   
6  6        7        329.520076     0.000000e+00            1.0   
7  7        8        265.402861     1.637766e-11            1.0   
8  8        9        308.464056     2.677219e+00            1.0   
9  9       10        385.144874     9.236861e+00            1.0   

   route_cost_mean  route_cost_std  success_rate_mean  success_rate_std  \
0         0.000000    0.000000e+00            1.00000          0.000000   
1         0.000000    0.000000e+00            0.00000          0.000000   
2       163.682620    2.995911e-14   